Perfect 👍 You want this **Face Recognition Assessment System integrated with Flask** so that:

* Student takes **MCQ assessment** in a Flask web app.
* Webcam is monitored in **real time**.
* If:

  * No face detected → Show `"Face not detected"`.
  * Multiple faces → `"Multiple faces detected"` + capture photo evidence.
  * Face turned (left, right, up, down) → `"Please look into the camera"`.
* After **10 violations**, exam auto-submits.

---

## 🔹 System Architecture

1. **Flask Backend**

   * Serves MCQ exam page.
   * Handles submission & scoring.

2. **Frontend (HTML + JS)**

   * Renders MCQs.
   * Starts webcam stream → sends frames to Flask (via AJAX/WebSocket).

3. **Face Monitoring Service (Flask API + OpenCV + MediaPipe)**

   * Receives frames.
   * Detects face count & head pose.
   * Returns JSON response (`status`, `fault_count`).

4. **Auto-submit Trigger**

   * If `fault_count >= 10` → frontend auto-submits exam form.

---

## 🔹 Folder Structure

```
flask_face_exam/
│── app.py                # Flask backend
│── static/
│    └── js/
│        └── camera.js    # Webcam + monitoring
│── templates/
│    └── exam.html        # MCQ exam page
```

---

## 🔹 Flask Backend (app.py)

```python
from flask import Flask, render_template, request, jsonify
import cv2
import mediapipe as mp
import numpy as np
import base64

app = Flask(__name__)

# Face mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

fault_count = 0
MAX_FAULTS = 10

def analyze_frame(image):
    global fault_count
    h, w, _ = image.shape
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    status = "Face OK"

    if results.multi_face_landmarks:
        if len(results.multi_face_landmarks) > 1:
            status = "Multiple faces detected!"
            fault_count += 1
        else:
            for landmarks in results.multi_face_landmarks:
                pitch, yaw, roll = get_head_pose(landmarks, w, h)
                if yaw > 20:
                    status = "Look into camera (Right)"
                    fault_count += 1
                elif yaw < -20:
                    status = "Look into camera (Left)"
                    fault_count += 1
                elif pitch > 15:
                    status = "Look into camera (Down)"
                    fault_count += 1
                elif pitch < -15:
                    status = "Look into camera (Up)"
                    fault_count += 1
    else:
        status = "Face not detected!"
        fault_count += 1

    return {"status": status, "faults": fault_count, "max_faults": MAX_FAULTS}


def get_head_pose(landmarks, frame_w, frame_h):
    face_2d, face_3d = [], []
    for idx, lm in enumerate(landmarks.landmark):
        if idx in [33, 263, 1, 61, 291, 199]:  # important points
            x, y = int(lm.x * frame_w), int(lm.y * frame_h)
            face_2d.append([x, y])
            face_3d.append([x, y, lm.z])

    face_2d = np.array(face_2d, dtype=np.float64)
    face_3d = np.array(face_3d, dtype=np.float64)

    focal_length = 1 * frame_w
    cam_matrix = np.array([[focal_length, 0, frame_w / 2],
                           [0, focal_length, frame_h / 2],
                           [0, 0, 1]])
    dist_matrix = np.zeros((4, 1), dtype=np.float64)
    success, rot_vec, _ = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
    rmat, _ = cv2.Rodrigues(rot_vec)
    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
    return angles


@app.route("/")
def home():
    return render_template("exam.html")


@app.route("/monitor", methods=["POST"])
def monitor():
    global fault_count
    data = request.json["frame"]
    img_data = base64.b64decode(data.split(",")[1])
    nparr = np.frombuffer(img_data, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    result = analyze_frame(frame)

    if result["faults"] >= MAX_FAULTS:
        result["submit"] = True
    else:
        result["submit"] = False

    return jsonify(result)


if __name__ == "__main__":
    app.run(debug=True)
```

---

## 🔹 Frontend (templates/exam.html)

```html
<!DOCTYPE html>
<html>
<head>
    <title>Online Exam</title>
    <script src="{{ url_for('static', filename='js/camera.js') }}"></script>
</head>
<body>
    <h2>MCQ Exam</h2>
    <form id="examForm" method="POST" action="/submit">
        <p>Q1. What is 2+2?</p>
        <input type="radio" name="q1" value="3"> 3
        <input type="radio" name="q1" value="4"> 4
        <input type="radio" name="q1" value="5"> 5
        <br><br>
        <p>Q2. Capital of India?</p>
        <input type="radio" name="q2" value="Delhi"> Delhi
        <input type="radio" name="q2" value="Mumbai"> Mumbai
        <input type="radio" name="q2" value="Chennai"> Chennai
        <br><br>
        <input type="submit" value="Submit">
    </form>

    <video id="video" width="320" height="240" autoplay></video>
    <p id="status"></p>
</body>
</html>
```

---

## 🔹 Webcam JS (static/js/camera.js)

```javascript
let video = document.getElementById("video");
let statusText = document.getElementById("status");
let canvas = document.createElement("canvas");
let ctx = canvas.getContext("2d");

navigator.mediaDevices.getUserMedia({ video: true })
.then(stream => {
    video.srcObject = stream;
});

setInterval(() => {
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    ctx.drawImage(video, 0, 0);

    let frame = canvas.toDataURL("image/jpeg");

    fetch("/monitor", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ frame: frame })
    })
    .then(res => res.json())
    .then(data => {
        statusText.innerText = data.status + " | Faults: " + data.faults;

        if (data.submit) {
            alert("❌ Too many faults! Auto-submitting exam.");
            document.getElementById("examForm").submit();
        }
    });
}, 2000); // check every 2 sec
```

---

✅ This setup will:

* Run an **MCQ exam in Flask**.
* Monitor webcam continuously.
* Detect **face presence, multiple faces, head direction**.
* After **10 warnings → auto-submit exam**.

---

Do you also want me to extend this so that **captured violation images (evidence)** are also **saved on the server & displayed in the report** after submission?


👉 Do you want me to also show live camera feed in the browser (instead of background monitoring) so students see what’s being tracked?